In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

data_hist_files = {'A': 'data_file_hist_all_cols', 
                   'B': 'data_file_hist_some_cols',
                   'C': 'data_file_hist_no_lkbk'}

data = []
for c, fname in data_hist_files.items():
    df_hist = pd.read_pickle(f'data/{fname}.pkl')
    test_cols = [d for d in df_hist.columns if d.endswith('test_scores')]
    other_cols = ['pos_labels', 'sim_num_trades', 'sim_max_profit']
    df = df_hist[other_cols + test_cols].copy()
    
    # calculate true positives and false positives
    for i,r in df.iterrows():
        relevant_elements = r[0]
        trades = r[1]
        profit = r[2]
        models = list(r.index)[3:]
        for m in models:
            if pd.isnull(r[m]):
                continue
            d = json.loads(r[m].replace("'",'"'))
            tp = int(d['rc'] * relevant_elements)
            fp = int(tp/d['pr']-tp) if d['pr'] != 0 else 0
            ratio = round(tp/fp,2) if fp>0 else 0
            act_trades = min(trades,tp)
            row = dict(c=c,
                       dataset=i,
                       classifer=m,
                       tp=tp,
                       fp=fp,
                       diff=tp-fp,
                       ratio=ratio,
                       pr=d.get('pr',0),
                       rc=d.get('rc',0),
                       pf=d.get('pf',0),
                       est_max_profit=(profit/trades)*act_trades if trades>0 else 0
                      )
            data.append(row)
df = pd.DataFrame(data).set_index(['c','dataset','classifer'])

In [18]:
rank1 = df['pr'].rank(ascending=False)              # Highest precision
rank2 = df['diff'].rank(ascending=False)            # Highest tp-fp difference
rank3 = df['ratio'].rank(ascending=False)           # Highest tp/fp ratio (similar to precision)
#rank4 = df['tp'].rank(ascending=False)              # Highest true positives (not good indicator)
#rank5 = df['est_max_profit'].rank(ascending=False)  # Highest estimated max profit

#ranks = [rank1,rank2,rank3,rank4,rank5]
ranks = [rank1,rank2,rank3]

df['score'] = sum(ranks)/len(ranks)                 # Average of ranks
df.sort_values(by='score').head(11)

tp    fp  diff  ratio      pr      rc      pf  \
c dataset   classifer                                                          
C 20210806i lrc_test_scores   453   277   176   1.64  0.6204  0.0082  1.3210   
A 20210806g abc_test_scores    98    13    85   7.54  0.8824  0.0031  1.0243   
B 20210806i lrc_test_scores   840   631   209   1.33  0.5708  0.0152  1.6651   
A 20210806g rfc_test_scores  2075  1856   219   1.12  0.5278  0.0656  1.2348   
  20210806i lrc_test_scores  1731  1535   196   1.13  0.5299  0.0313  1.0503   
C 20210806d lrc_test_scores    40    26    14   1.54  0.6000  0.0009  1.0045   
            gbc_test_scores    58    38    20   1.53  0.6000  0.0013  1.0126   
B 20210806i gbc_test_scores  2710  2460   250   1.10  0.5241  0.0490  0.6239   
A 20210806i gbc_test_scores  1919  1770   149   1.08  0.5201  0.0347  0.8400   
  20210806a abc_test_scores    60    54     6   1.11  0.5263  0.0016  0.9951   
  20210806i abc_test_scores  3534  3399   135   1.04  0.5097  0.0639  0.5109   

                             est_max_profit      score  
c dataset   classifer                                   
C 20210806i lrc_test_scores     4316.470906   3.666667  
A 20210806g abc_test_scores      288.939732   4.000000  
B 20210806i lrc_test_scores     8004.052011   5.000000  
A 20210806g rfc_test_scores     1084.998178   6.000000  
  20210806i lrc_test_scores     9843.078247   6.000000  
C 20210806d lrc_test_scores      614.999094   6.166667  
            gbc_test_scores      891.748687   6.166667  
B 20210806i gbc_test_scores     9843.078247   7.000000  
A 20210806i gbc_test_scores     9843.078247   9.333333  
  20210806a abc_test_scores      673.231226   9.833333  
  20210806i abc_test_scores     9843.078247  10.333333

In [22]:
df.sort_values(by='score').reset_index()[['classifer', 'dataset','c','tp','fp','diff','ratio','pr','rc','pf']].head(11)

,classifer,dataset,c,tp,fp,diff,ratio,pr,rc,pf
0,lrc_test_scores,20210806i,C,453,277,176,1.64,0.6204,0.0082,1.3210
1,abc_test_scores,20210806g,A,98,13,85,7.54,0.8824,0.0031,1.0243
2,lrc_test_scores,20210806i,B,840,631,209,1.33,0.5708,0.0152,1.6651
3,rfc_test_scores,20210806g,A,2075,1856,219,1.12,0.5278,0.0656,1.2348
4,lrc_test_scores,20210806i,A,1731,1535,196,1.13,0.5299,0.0313,1.0503
5,lrc_test_scores,20210806d,C,40,26,14,1.54,0.6000,0.0009,1.0045
6,gbc_test_scores,20210806d,C,58,38,20,1.53,0.6000,0.0013,1.0126
7,gbc_test_scores,20210806i,B,2710,2460,250,1.10,0.5241,0.0490,0.6239
8,gbc_test_scores,20210806i,A,1919,1770,149,1.08,0.5201,0.0347,0.8400
9,abc_test_scores,20210806a,A,60,54,6,1.11,0.5263,0.0016,0.9951


In [21]:
df.sort_values(by='pf', ascending=False)

tp     fp   diff  ratio      pr      rc  \
c dataset   classifer                                                     
B 20210806i lrc_test_scores    840    631    209   1.33  0.5708  0.0152   
C 20210806e mlp_test_scores    156    400   -244   0.39  0.2805  0.0092   
  20210806i lrc_test_scores    453    277    176   1.64  0.6204  0.0082   
A 20210806d lrc_test_scores    308    393    -85   0.78  0.4393  0.0069   
  20210806h rfc_test_scores    813   1167   -354   0.70  0.4106  0.0307   
B 20210806g rfc_test_scores   3287   3631   -344   0.91  0.4751  0.1039   
  20210806d lrc_test_scores     98    124    -26   0.79  0.4412  0.0022   
A 20210806g rfc_test_scores   2075   1856    219   1.12  0.5278  0.0656   
C 20210806g mlp_test_scores   1452   2194   -742   0.66  0.3982  0.0459   
  20210806e gnb_test_scores     88    385   -297   0.23  0.1857  0.0052   
A 20210806g xgb_test_scores   2467   3613  -1146   0.68  0.4057  0.0780   
C 20210806g gnb_test_scores   1059   2144  -1085   0.49  0.3306  0.0335   
  20210806h rfc_test_scores   1515   2263   -748   0.67  0.4010  0.0572   
B 20210806e rfc_test_scores     61    169   -108   0.36  0.2647  0.0036   
C 20210806f mlp_test_scores     55    281   -226   0.20  0.1633  0.0036   
  20210806e rfc_test_scores    108    378   -270   0.29  0.2222  0.0064   
B 20210806h rfc_test_scores   1504   1852   -348   0.81  0.4481  0.0568   
  20210806f xgb_test_scores    137    369   -232   0.37  0.2703  0.0090   
  20210806e mlp_test_scores    444   1953  -1509   0.23  0.1852  0.0261   
C 20210806h xgb_test_scores   1528   3010  -1482   0.51  0.3367  0.0577   
B 20210806h xgb_test_scores   1438   2390   -952   0.60  0.3756  0.0543   
C 20210806g rfc_test_scores   3309   4687  -1378   0.71  0.4138  0.1046   
A 20210806i lrc_test_scores   1731   1535    196   1.13  0.5299  0.0313   
  20210806e xgb_test_scores    149    771   -622   0.19  0.1618  0.0088   
  20210806k lrc_test_scores     21    141   -120   0.15  0.1290  0.0011   
B 20210806e xgb_test_scores    355    819   -464   0.43  0.3023  0.0209   
A 20210806f abc_test_scores      0      0      0   0.00  0.0000  0.0000   
  20210806g abc_test_scores     98     13     85   7.54  0.8824  0.0031   
C 20210806e xgb_test_scores    575    725   -150   0.79  0.4421  0.0338   
B 20210806f rfc_test_scores     27    128   -101   0.21  0.1739  0.0018   
C 20210806k lrc_test_scores      0      0      0   0.00  0.0000  0.0000   
A 20210806d abc_test_scores    129    212    -83   0.61  0.3774  0.0029   
  20210806b lrc_test_scores     19     69    -50   0.28  0.2143  0.0008   
C 20210806f rfc_test_scores     55    151    -96   0.36  0.2667  0.0036   
A 20210806e abc_test_scores      0      0      0   0.00  0.0000  0.0000   
C 20210806d gbc_test_scores     58     38     20   1.53  0.6000  0.0013   
A 20210806b gbc_test_scores     11     22    -11   0.50  0.3333  0.0005   
B 20210806g abc_test_scores      6      0      6   0.00  1.0000  0.0002   
A 20210806l lrc_test_scores     28    112    -84   0.25  0.2000  0.0011   
C 20210806k gbc_test_scores      5      0      5   0.00  1.0000  0.0003   
  20210806d lrc_test_scores     40     26     14   1.54  0.6000  0.0009   
  20210806a lrc_test_scores     30    114    -84   0.26  0.2083  0.0008   
A 20210806k gbc_test_scores     11     33    -22   0.33  0.2500  0.0006   
  20210806f xgb_test_scores    192    377   -185   0.51  0.3373  0.0126   
B 20210806k lrc_test_scores      0      0      0   0.00  0.0000  0.0000   
C 20210806l gbc_test_scores      5     10     -5   0.50  0.3333  0.0002   
A 20210806h lrc_test_scores     26     39    -13   0.67  0.4000  0.0010   
C 20210806k mlp_test_scores     11     49    -38   0.22  0.1818  0.0006   
  20210806g abc_test_scores      0      0      0   0.00  0.0000  0.0000   
  20210806f abc_test_scores      0      0      0   0.00  0.0000  0.0000   
B 20210806c abc_test_scores      0      0      0   0.00  0.0000  0.0000   
  20210806l abc_test_scores      0      0      0   0.00  

In [15]:
# df[df.pf>1].sort_values(by='pf', ascending=False)

df['rank1'] = df['diff'].rank(ascending=False)            # Highest tp-fp difference
df['rank2'] = df['pr'].rank(ascending=False)              # Highest precision
df['rank3'] = df['ratio'].rank(ascending=False)           # Highest tp/fp ratio
df['rank4'] = df['tp'].rank(ascending=False)              # Highest true positives
df['rank5'] = df['est_max_profit'].rank(ascending=False)  # Highest estimated max profit


display(df[df.pf>1].describe())
display(df[df.pf<=1].describe())

,tp,fp,diff,ratio,pr,rc,pf,est_max_profit,score,rank1,rank2,rank3,rank4,rank5
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,557.854167,823.979167,-266.125000,0.676458,0.356054,0.019000,1.096977,1016.072080,94.652778,95.822917,90.468750,97.666667,137.770833,140.427083
std,861.602268,1155.394940,439.462177,1.092955,0.223421,0.027627,0.131353,1936.675398,51.086607,54.536781,70.961063,73.148300,44.980842,44.380127
min,0.000000,0.000000,-1509.000000,0.000000,0.000000,0.000000,1.000700,0.000000,3.666667,2.000000,1.500000,1.000000,56.000000,42.000000
25%,24.750000,38.750000,-345.000000,0.230000,0.206225,0.000875,1.012375,75.059917,59.458333,46.500000,29.750000,36.125000,94.750000,111.500000
50%,103.000000,279.000000,-90.500000,0.495000,0.335000,0.004400,1.042600,276.475821,90.000000,104.500000,79.500000,81.000000,148.250000,150.500000
75%,819.750000,1259.000000,0.000000,0.727500,0.441425,0.030850,1.123975,1084.998178,139.208333,132.500000,157.625000,162.000000,163.875000,168.875000
max,3309.000000,4687.000000,219.000000,7.540000,1.000000,0.104600,1.665100,9843.078247,169.333333,180.000000,220.500000,219.500000,220.500000,220.500000


,tp,fp,diff,ratio,pr,rc,pf,est_max_profit,score,rank1,rank2,rank3,rank4,rank5
count,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,3170.313725,6957.441176,-3787.127451,0.332451,0.215034,0.091053,0.718459,4097.526939,133.993464,133.718137,134.977941,133.284314,123.848039,123.223039
std,5623.007219,12199.495035,7774.428332,0.289320,0.163196,0.152820,0.330087,5688.158730,39.718288,74.079376,70.117904,70.487169,77.174848,77.092312
min,0.000000,0.000000,-42257.000000,0.000000,0.000000,0.000000,0.018300,0.000000,7.000000,1.000000,10.000000,8.000000,1.000000,6.500000
25%,0.000000,0.000000,-2910.750000,0.000000,0.000000,0.000000,0.468525,0.000000,107.416667,46.500000,73.750000,69.500000,51.750000,52.750000
50%,312.500000,819.000000,-415.000000,0.330000,0.250000,0.012050,0.897450,798.993960,150.250000,140.500000,134.500000,133.000000,119.500000,120.500000
75%,3700.500000,7481.750000,0.000000,0.550000,0.353300,0.110825,0.995200,8443.794990,162.166667,201.250000,220.500000,219.500000,220.500000,220.500000
max,30871.000000,50531.000000,250.000000,1.110000,0.526300,0.561500,1.000000,17342.974458,204.666667,252.000000,220.500000,219.500000,220.500000,220.500000


In [4]:
df_hist

,use_atr,ratio,reverse,window,length,pos_labels,imbalance,train_imbal,test_imbal,sim_init_val,sim_fee_buy,sim_fee_sell,sim_fee_per,sim_num_trades,sim_max_profit,sim_bad_trades,gnb_train_scores,gnb_test_scores,lrc_train_scores,lrc_test_scores,rfc_train_scores,rfc_test_scores,abc_train_scores,abc_test_scores,gbc_train_scores,gbc_test_scores,xgb_train_scores,xgb_test_scores,mlp_train_scores,mlp_test_scores
suffix,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20210806a,False,"(0.01, 0.005)",True,30,141174,37979,0.269023,0.262033,0.319201,1.0,0.0,0.0,0.1,1179.0,13228.993600,0.0,"{'f1': 0.0377, 'pr': 0.3764, 'rc': 0.0198, 'pf...","{'f1': 0.0217, 'pr': 0.3077, 'rc': 0.0113, 'pf...","{'f1': 0.0001, 'pr': 0.1818, 'rc': 0.0001, 'pf...","{'f1': 0.0016, 'pr': 0.2083, 'rc': 0.0008, 'pf...","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0, 'pf': 109462...","{'f1': 0.211, 'pr': 0.3804, 'rc': 0.146, 'pf':...","{'f1': 0.0096, 'pr': 0.6683, 'rc': 0.0048, 'pf...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.0153, 'pr': 0.9636, 'rc': 0.0077, 'pf...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.588, 'pr': 0.9309, 'rc': 0.4298, 'pf'...","{'f1': 0.1685, 'pr': 0.3605, 'rc': 0.11, 'pf':...","{'f1': 0.1603, 'pr': 0.5985, 'rc': 0.0926, 'pf...","{'f1': 0.1116, 'pr': 0.4145, 'rc': 0.0645, 'pf..."
20210806b,False,"(0.01, 0.0025)",True,30,141174,23782,0.168459,0.169634,0.196954,1.0,0.0,0.0,0.1,1093.0,6620.481356,0.0,"{'f1': 0.0203, 'pr': 0.2266, 'rc': 0.0106, 'pf...","{'f1': 0.0148, 'pr': 0.2586, 'rc': 0.0076, 'pf...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 0.9955}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0, 'pf': 248910...","{'f1': 0.038, 'pr': 0.3008, 'rc': 0.0203, 'pf'...","{'f1': 0.0035, 'pr': 1.0, 'rc': 0.0017, 'pf': ...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.0044, 'pr': 0.975, 'rc': 0.0022, 'pf'...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.3172, 'pr': 0.9575, 'rc': 0.1901, 'pf...","{'f1': 0.0153, 'pr': 0.265, 'rc': 0.0079, 'pf'...","{'f1': 0.0469, 'pr': 0.5709, 'rc': 0.0245, 'pf...","{'f1': 0.0255, 'pr': 0.4031, 'rc': 0.0132, 'pf..."
20210806c,False,"(0.0075, 0.0025)",True,30,141174,29824,0.211257,0.220094,0.233808,1.0,0.0,0.0,0.1,1456.0,3126.592203,0.0,"{'f1': 0.2332, 'pr': 0.2506, 'rc': 0.218, 'pf'...","{'f1': 0.182, 'pr': 0.2573, 'rc': 0.1408, 'pf'...","{'f1': 0.0001, 'pr': 0.5, 'rc': 0.0, 'pf': 1.001}","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0, 'pf': 251641...","{'f1': 0.042, 'pr': 0.3302, 'rc': 0.0224, 'pf'...","{'f1': 0.005, 'pr': 0.8169, 'rc': 0.0025, 'pf'...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.0063, 'pr': 0.9481, 'rc': 0.0032, 'pf...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.3233, 'pr': 0.9507, 'rc': 0.1948, 'pf...","{'f1': 0.0222, 'pr': 0.2857, 'rc': 0.0115, 'pf...","{'f1': 0.032, 'pr': 0.654, 'rc': 0.0164, 'pf':...","{'f1': 0.0117, 'pr': 0.28, 'rc': 0.006, 'pf': ..."
20210806d,True,"(2, 1)",True,15,141174,44769,0.317119,0.317522,0.341973,1.0,0.0,0.0,0.1,1128.0,17342.974458,1.0,"{'f1': 0.0816, 'pr': 0.3498, 'rc': 0.0462, 'pf...","{'f1': 0.0753, 'pr': 0.3689, 'rc': 0.0419, 'pf...","{'f1': 0.0002, 'pr': 0.2308, 'rc': 0.0001, 'pf...","{'f1': 0.0017, 'pr': 0.6, 'rc': 0.0009, 'pf': ...","{'f1': 1.0, 'pr': 1.0, 'rc': 1.0, 'pf': 185098...","{'f1': 0.1975, 'pr': 0.3904, 'rc': 0.1322, 'pf...","{'f1': 0.0081, 'pr': 0.6476, 'rc': 0.0041, 'pf...","{'f1': 0.0, 'pr': 0.0, 'rc': 0.0, 'pf': 1}","{'f1': 0.0229, 'pr': 0.8178, 'rc': 0.0116, 'pf...","{'f1': 0.0026, 'pr': 0.6, 'rc': 0.0013, 'pf': ...","{'f1': 0.534, 'pr': 0.8965, 'rc': 0.3802, 'pf'...","{'f1': 0.1707, 'pr': 0.3983, 'rc': 0.1086, 'pf...","{'f1': 0.219, 'pr': 0.5916, 'rc': 0.1344, 'pf'...","{'f1': 0.1145, 'pr': 0.4266, 'rc': 0.0662, 'pf..."
20210806e,True,"(4, 2)",True,15,141174,17024,0.120589,0.123430,0.124245,1.0,0.0,0.0,0.1,335.0,416.560613,0.0,"{'f1': 0.0139, 'pr': 0.1601, 'rc': 0.0073, 'pf...","{'f1': 0.0102, 'pr': 0.1857, 'rc': 0.0052, 'pf...","{'f1': 0.0, 'pr': 0.0

# Build Best Performers

In [5]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

import pickle
import datetime
import re

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

def train_test_split(X, y, train_idx=None, test_idx=None):
    X_train = X.loc[train_idx]
    y_train = y.loc[train_idx]
    X_test = X.loc[test_idx]
    y_test = y.loc[test_idx]
    return (X_train, y_train, X_test, y_test)

def load_split_data(suffix, split=False):
    X = pd.read_pickle(f'data/X_{suffix}.pkl')
    y = pd.read_pickle(f'data/y_{suffix}.pkl')
    if split:
        X_train, y_train, X_test, y_test = train_test_split(X, y, X.loc['2018':'2020'].index, X.loc['2021':].index)
        return X_train, y_train, X_test, y_test
    else:
        return X, y

def get_columns(X,lookbacks):
    # Drop columns with lookbacks equal to or greater than X
    columns = list(X.columns)
    for c in X.columns:
        if m := re.match(r'^.*_([0-9]+)$', c):
            if int(m[1]) > lookbacks:
                columns.remove(c)
    return columns

clfs = {
    'gnb' : GaussianNB(),
    'lrc' : LogisticRegression(random_state=42, max_iter=10000),
    'rfc' : RandomForestClassifier(random_state=42, n_jobs=-1),
    'abc' : AdaBoostClassifier(random_state=42),
    'gbc': GradientBoostingClassifier(random_state=42),
    'xgb' : xgb.XGBClassifier(n_jobs=-1, random_state=42, use_label_encoder=False),
    'mlp' : make_pipeline(StandardScaler(),MLPClassifier(random_state=42)),
}
numlkb_dict = {'A': 15, 
               'B': 3,
               'C': 0}

In [6]:
dataset = '20210806i'
ncols = 'A'
cname = 'abc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

Fitting model...


KeyboardInterrupt: 

In [ ]:
dataset = '20210806i'
ncols = 'B'
cname = 'gbc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

In [ ]:
dataset = '20210806i'
ncols = 'A'
cname = 'lrc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

In [ ]:
dataset = '20210806i'
ncols = 'A'
cname = 'gbc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

In [ ]:
dataset = '20210806i'
ncols = 'B'
cname = 'lrc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

In [ ]:
dataset = '20210806i'
ncols = 'C'
cname = 'lrc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

In [ ]:
dataset = '20210806g'
ncols = 'A'
cname = 'rfc'

print('Fitting model...')
X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
columns = get_columns(X_train, numlkb_dict[ncols])
clfs[cname].fit(X_train[columns],y_train)

timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
filename = f"models/nm_{cname}_{ncols}_{dataset}_{timestamp}.pkl"
print(f'Saving model to {filename}...')
pickle.dump(clfs[cname], open(filename, 'wb'))

# Validate Built Models

In [ ]:
models = [
    'models/nm_abc_A_20210806i_2108100754.pkl',
    'models/nm_gbc_B_20210806i_2108100759.pkl',
    'models/nm_lrc_A_20210806i_2108100800.pkl',
    'models/nm_gbc_A_20210806i_2108100813.pkl',
    'models/nm_lrc_B_20210806i_2108100814.pkl',
    'models/nm_lrc_C_20210806i_2108100814.pkl',
    'models/nm_rfc_A_20210806g_2108100814.pkl',
]

numlkb_dict = {'A': 15, 
               'B': 3,
               'C': 0}

def score_clf(clf, X, y):
    f1 = round(f1_score(y,clf.predict(X)),4)
    pr = round(precision_score(y,clf.predict(X)),4)
    rc = round(recall_score(y,clf.predict(X)),4)
    return {'f1':f1, 'pr':pr, 'rc':rc}

for m in models:
    dataset = m[16:25]
    ncols = m[14]
    
    X_train, y_train, X_test, y_test = load_split_data(dataset, split=True)
    columns = get_columns(X_train, numlkb_dict[ncols])
    clf = pickle.load(open(m, 'rb'))
    
    scores = score_clf(clf, X_test[columns], y_test)
    print(f"{m}:  {scores}")